In [9]:
from pyiem.plot import get_cmap, figure, plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.colors as mpcolors
import datetime
import pytz
from zoneinfo import ZoneInfo
from pandas.io.sql import read_sql
from pyiem.util import get_sqlalchemy_conn
from sqlalchemy import text
cmap = get_cmap("gist_rainbow")
cmap.set_under("tan")
norm = mpcolors.BoundaryNorm(
    list(range(11)), 256
)


In [35]:
with get_sqlalchemy_conn("iem") as conn:
    df = read_sql("""SELECT valid, state, tmpf, ugc_zone,
     id from current_log c JOIN stations t ON (t.iemid = c.iemid)
     WHERE t.network in ('IA_ASOS') and valid > '2024-01-20 00:00' and valid < '2024-01-24 21:00'
     and tmpf is not null 
     ORDER by valid ASC
     """, conn, index_col=None)

with get_sqlalchemy_conn("mos") as conn:
    mosdf = read_sql(
        text("""
    SELECT ftime, tmp, model from t2024 where ftime > '2024-01-24 20:00' and
    runtime > '2024-01-24' and station = ANY(:sts) and tmp is not null and model in ('GFS', 'NBE', 'NBS')
        """),
        conn,
        params={"sts": [f"K{sid}" for sid in df["id"].unique()]},
    )
print(mosdf.groupby("model").std())                     

                          ftime       tmp
model                                    
GFS   0 days 17:21:32.030597812  2.494061
NBE   3 days 00:03:21.120355927  4.814064
NBS   0 days 18:38:03.288761998  1.597057


In [23]:
with get_sqlalchemy_conn("postgis") as conn:
    warndf = read_sql("""SELECT issue, expire, ugc from warnings_2022 where issue > '2022-04-20 12:00'
    and issue < '2022-04-22' and substr(ugc, 1, 2) = 'IA' and phenomena = 'FG' and significance = 'Y'
     """, conn, index_col=None)

In [24]:
df2 = df.merge(warndf, left_on="ugc_zone", right_on="ugc", how="left")
df2["warned"] = (df2["issue"] < df2["valid"]) & (df2["expire"] > df2["valid"])
df2

,valid,vsby,ugc_zone,id,issue,expire,ugc,warned
0,2022-04-20 17:04:00+00:00,3.0,IAZ301,SPW,NaT,NaT,NaN,False
1,2022-04-20 17:04:00+00:00,4.0,IAZ035,FOD,2022-04-21 04:16:00+00:00,2022-04-21 13:51:00+00:00,IAZ035,False
2,2022-04-20 17:05:00+00:00,2.5,IAZ060,DSM,2022-04-21 04:16:00+00:00,2022-04-21 16:00:00+00:00,IAZ060,False
3,2022-04-20 17:05:00+00:00,10.0,IAZ029,OLZ,NaT,NaT,NaN,False
4,2022-04-20 17:05:00+00:00,10.0,IAZ090,SDA,2022-04-21 04:08:00+00:00,2022-04-21 16:00:00+00:00,IAZ090,False
...,...,...,...,...,...,...,...,...
36390,2022-04-22 03:15:00+00:00,10.0,IAZ040,IIB,NaT,NaT,NaN,False
36391,2022-04-22 03:15:00+00:00,10.0,IAZ074,OXV,2022-04-21 08:31:00+00:00,2022-04-21 16:00:00+00:00,IAZ074,False
36392,2022-04-22 03:15:00+00:00,10.0,IAZ018,CCY,2022-04-21 07:59:00+00:00,2022-04-21 13:00:00+00:00,IAZ018,False
36393,2022-04-22 03:15:00+00:00,10.0,IAZ037,IFA,2022-04-21 08:31:00+00:00,2022-04-21 13:51:00+00:00,IAZ037,False


In [36]:
x = []
y = []
base = datetime.datetime(2024, 1, 23, 6).replace(tzinfo=pytz.utc)
for hr in range(24*6):
    sts = base + datetime.timedelta(hours=hr)
    ets = sts + datetime.timedelta(hours=1)
    df2 = df[(df['valid'] >= sts) & (df['valid'] < ets)]
    if df2.empty:
        continue
    x.append(sts)
    y.append(df2['tmpf'].max() - df2['tmpf'].min())
mosx = []
mosy = []
base = datetime.datetime(2024, 1, 25, 0).replace(tzinfo=pytz.utc)
for hr in range(24*6):
    sts = base + datetime.timedelta(hours=hr)
    ets = sts + datetime.timedelta(hours=1)
    df2 = mosdf[(mosdf['ftime'] >= sts) & (mosdf['ftime'] < ets)]
    if df2.empty:
        continue
    mosx.append(sts)
    mosy.append(df2['tmp'].max() - df2['tmp'].min())


In [38]:
sts = datetime.datetime(2024, 1, 23, 0, tzinfo=ZoneInfo("America/Chicago"))
ets = datetime.datetime(2024, 1, 29, 0, tzinfo=ZoneInfo("America/Chicago"))
interval = datetime.timedelta(hours=12)
xticks = []
xticklabels = []
now = sts
while now <= ets:
    if now.hour % 3 != 0:
        now += interval
        continue
    fmt = "%-I %p" if now.hour != 0 else '%-I %p\n%-d %b'
    xticks.append(now)
    xticklabels.append(now.strftime(fmt))
    now += interval

fig = figure(
    title="Iowa Airport Air Temperatures Obs + Forecasts",
    subtitle="Forecasts based on available GFS+NBM Model Output Statistics (MOS)",
    figsize=(8, 6))
ax = fig.add_axes([0.1, 0.56, 0.8, 0.33])
ax.scatter(df["valid"], df["tmpf"], color='b', label="Observed")
ax.scatter(mosdf["ftime"], mosdf["tmp"], color='r', label="MOS Forecast")
ax.set_xticks([])
ax.set_ylim(20, 50)
ax.set_ylabel("Air Temperatures $^\circ$F")
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels)
ax.set_xlim(sts, ets)
ax.legend()
ax.grid(True)
ax.set_xlabel("Central Standard Time")

ax = fig.add_axes([0.1, 0.1, 0.8, 0.33])
ax.plot(x, y, color='b', label="Observed")
ax.plot(mosx, mosy, color='r', label="MOS Forecast")
ax.grid(True)
ax.legend()
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels)
ax.set_xlim(sts, ets)
ax.set_xlabel("Central Standard Time")
ax.set_ylabel(r"Hourly Temp Range $^\circ$F")

fig.savefig('240125.png')
plt.close()